## Binance Data Preparation

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import configparser
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np

In [ ]:
config_data=configparser.ConfigParser()
config_data.read("config.ini")
keys=config_data['keys']
client=Client(keys['apiKey'],keys['secretKey'])
print('logged in')

In [ ]:
tickers=client.get_all_tickers()
tickers

In [ ]:
ticker_df=pd.DataFrame(tickers)
ticker_df

In [ ]:
ticker_df.set_index('symbol',inplace=True)

In [ ]:
ticker_df

In [ ]:
depth=client.get_order_book(symbol='ETHBTC')
depth
depth_df=pd.DataFrame(depth['asks'])
depth_df.columns=['Price','Volume']
depth_df.head()

In [ ]:
#  [
#   [
#     1499040000000,      // Open time
#     "0.01634790",       // Open
#     "0.80000000",       // High
#     "0.01575800",       // Low
#     "0.01577100",       // Close
#     "148976.11427815",  // Volume
#     1499644799999,      // Close time
#     "2434.19055334",    // Quote asset volume
#     308,                // Number of trades
#     "1756.87402397",    // Taker buy base asset volume
#     "28.46694368",      // Taker buy quote asset volume
#     "17928899.62484339" // Ignore.
#   ]
# ]
client.get_historical_klines??

In [ ]:
historical = client.get_historical_klines('BTCUSDT',Client.KLINE_INTERVAL_1DAY,'17 Aug 2017')# parameters:(symbol, interval, date_from)
historical

btcusdt is the price of bitcion relative to the US dollar; 1 usdt ~ 1 dollar; usdt was chosen because its price is tied to the dollar and does not fluctuate like other cryptocurrencies

In [ ]:
hist_df=pd.DataFrame(historical)
hist_df.head()

In [ ]:
hist_df.columns=['Open Time','Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of Trades','TB Base Volume','TB Quote Volume','Ignore']
hist_df.head()

In [ ]:
hist_df.shape

In [ ]:
hist_df.dtypes #int64 + objects (which are strings) >> preprocessing

In [ ]:
#convert from Unix timestamps to date
hist_df['Open Time']= pd.to_datetime(hist_df['Open Time']/1000, unit='s')
hist_df['Close Time']= pd.to_datetime(hist_df['Close Time']/1000, unit='s')
hist_df.dtypes

In [ ]:
hist_df.head() #notice open time and close time 

Binance was launched in July 2017 and therefore the historical data for trading pairs listed on Binance is available from the time the exchange began operations.

In [ ]:
numeric_columns=['Open','High','Low','Close','Volume','Quote Asset Volume','TB Base Volume','TB Quote Volume']
hist_df[numeric_columns]=hist_df[numeric_columns].apply(pd.to_numeric,axis=1)
hist_df.head()

In [ ]:
hist_df.dtypes

In [ ]:
hist_df.describe()

In [ ]:
hist_df.describe(include='object')

In [ ]:
hist_df.describe(include='datetime')

In [ ]:
hist_df.info()

In [ ]:
hist_df.set_index('Open Time', inplace=True)

In [ ]:
hist_df['Weighted Avg Price'] = hist_df['Quote Asset Volume'] / hist_df['Volume'] #SIMILAR METRIC TO BPI

In [ ]:
hist_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
correlations = hist_df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlations, annot=True, cmap="coolwarm", vmin=-1, vmax=1, linewidths=.5)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
data_csv=hist_df.drop(['Quote Asset Volume','TB Quote Volume','Number of Trades','TB Base Volume','Close Time','Ignore'],axis=1)
data_csv.to_csv('btc_prediction_data.csv')
data_csv

## Blockchain Data Preparation from Blockchain.com

In [ ]:

features = [
    "avg-block-size",
    "blockchain-size",
    "blocks-size",
    "cost-per-transaction",
    "cost-per-transaction-percent",
    "difficulty",
    "estimated-transaction-volume",
    "estimated-transaction-volume-usd",
    "hash-rate",
    "n-transactions-total",
    "market-price",
    "n-unique-addresses",
    "n-transactions-per-block",
    "n-transactions",
    "miners-revenue",
    "transaction-fees",
    "transaction-fees-usd",
    "n-transactions-excluding-popular",
    "n-transactions-excluding-chains-longer-than-100",
    "output-volume",
    "total-bitcoins",
    "trade-volume"
]

In [ ]:
def fetch_data(feature):
    url = f"https://api.blockchain.info/charts/market-price?timespan=6years&format=json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data["values"]
    else:
        print(f"Error fetching {feature}: {response.status_code}")
        return []

In [ ]:
dataframes = []
for feature in features:
    data = fetch_data(feature)
    
    if data:
        df = pd.DataFrame(data)
        df["feature"] = feature
        dataframes.append(df)

blockchainData_df = pd.concat(dataframes, ignore_index=True)
blockchainData_df.rename(columns={"x": "timestamp", "y": "value"}, inplace=True)

print(blockchainData_df)

In [ ]:
blockchainData_df["timestamp"] = pd.to_datetime(blockchainData_df["timestamp"], unit="s")
blockchainData_df

In [ ]:
blockchainData_df.info()

In [ ]:
blockchainData_df.feature.unique()

In [ ]:
blockchainData_df.set_index('timestamp')

In [ ]:
pivoted_df = blockchainData_df.pivot_table(index="timestamp", columns="feature", values="value")

In [ ]:
pivoted_df

In [ ]:
pivoted_df.info()

In [ ]:
pivoted_df.describe()

In [ ]:
num_years = (blockchainData_df["timestamp"].max() - blockchainData_df["timestamp"].min()).total_seconds() / (365.25 * 24 * 60 * 60)
num_years 

## Merging the Data

In [ ]:
mergedData_df = pd.merge(hist_df, pivoted_df, left_index=True, right_index=True)
mergedData_df                     

In [ ]:
mergedData_df.info()

In [ ]:
correlations = mergedData_df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlations, annot=True, cmap="coolwarm", vmin=-1, vmax=1, linewidths=.5)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
correlations['market-price']

In [ ]:
mergedData_df.columns

In [ ]:
mergedData_df.drop(columns=['Quote Asset Volume', 'Number of Trades', 'TB Base Volume',
       'TB Quote Volume', 'Ignore','Close Time'],inplace=True)

## technical indicators 

In [ ]:
#calculate the EMA
mergedData_df['EMA'] = mergedData_df['Close'].ewm(span=12).mean()
mergedData_df.head()

In [ ]:
#calculate the MACD
temp_df=pd.DataFrame()
temp_df['EMA_12'] = mergedData_df['Close'].ewm(span=12).mean()
temp_df['EMA_26'] = mergedData_df['Close'].ewm(span=26).mean()
mergedData_df['MACD'] = temp_df['EMA_12'] - temp_df['EMA_26']
mergedData_df

In [ ]:
#Calculate the RSI
price_changes = mergedData_df['Close'].diff()
positive_changes = price_changes.where(price_changes > 0, 0)
negative_changes = -price_changes.where(price_changes < 0, 0)
temp_df['avg_positive_changes'] = positive_changes.rolling(window=14).mean()
temp_df['avg_negative_changes'] = negative_changes.rolling(window=14).mean()
temp_df['RS'] = temp_df['avg_positive_changes'] / temp_df['avg_negative_changes']#NaN 
mergedData_df['RSI'] = 100 - (100 / (1 + temp_df['RS']))
mergedData_df.info() #null values 

In [ ]:
price_changes.where(price_changes>0,0).rolling(window=14).mean()

In [ ]:
N = 10  # You can set the desired N value for the momentum calculation
mergedData_df['MOM'] = mergedData_df['Close'] - mergedData_df['Close'].shift(N)
mergedData_df.info() #new null values 

In [ ]:
null_indices = mergedData_df[mergedData_df.isnull().any(axis=1)].reset_index()["index"].tolist()
null_indices #13 rows, rolling window is of size 14 

In [ ]:
N = 14  # You can set the desired N value for the PROC calculation
mergedData_df['PROC'] = (mergedData_df['Close'] - mergedData_df['Close'].shift(N)) / mergedData_df['Close'].shift(N) * 100
mergedData_df.info()

In [ ]:
N = 14  # You can set the desired N value for the %K calculation
low_N = mergedData_df['Low'].rolling(window=N).min()
high_N = mergedData_df['High'].rolling(window=N).max()
mergedData_df['%K'] = (mergedData_df['Close'] - low_N) / (high_N - low_N) * 100
mergedData_df.info()

In [ ]:
sub_df=mergedData_df[['EMA','MACD','RSI','MOM','PROC','%K','market-price']]
correlations = sub_df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlations, annot=True, cmap="coolwarm", vmin=-1, vmax=1, linewidths=.5)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
csv_file=mergedData_df.to_csv('data_with_tecInd.csv')
csv_file

In [ ]:
mergedData_df.dropna(inplace=True)

In [ ]:
csv_file=mergedData_df.to_csv('data_with_tecInd.csv')
csv_file

In [ ]:
mergedData_df